In [68]:
import pandas as pd
import google.generativeai as genai
import os
from dotenv import load_dotenv
from function import *
load_dotenv()


True

# データの読み込み

In [48]:
df = pd.read_excel("Day1_アンケート .xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 22 columns):
 #   Column                                                                  Non-Null Count  Dtype 
---  ------                                                                  --------------  ----- 
 0   申込区分                                                                    2996 non-null   object
 1   申込区分（日本語）                                                               2996 non-null   object
 2   ロール                                                                     2996 non-null   object
 3   本日の総合的な満足度を５段階で教えてください。                                                 2996 non-null   int64 
 4   本日の講義内容について５段階で教えてください。 
学習量は適切だった                                      2996 non-null   int64 
 5   本日の講義内容について５段階で教えてください。 
講義内容が十分に理解できた                                  2996 non-null   int64 
 6   本日の講義内容について５段階で教えてください。 
運営側のアナウンスが適切だった                                2996 non-null   

In [49]:
column_number = [str(i+1) for i in range (df.shape[1])]
df.columns = column_number
df.head()

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,OTHER,不明,student,2,3,3,2,2,4,2,...,3,1,5,並列で学習させる際の種類について,半導体の話がおもしろそうだと思えたこと,OmniCampusにZoomのリンクを載せる動線では駄目でしょうか？_毎回、動線が散漫とし...,NaN,松尾研がどのような研究をされているのか知りたいです。,昨年資料にもありましたが、第１回目で各回の紹介は要らないように思います。,ぜひ英語表記を併記してほしいです！日本語表記になると訳文だとわかる日本語になってしまい、本当...
1,OTHER,不明,student,2,2,2,2,1,1,3,...,4,3,3,主に大規模言語モデル(LLM)の概要や講義の前半について学びました。LLMは単語列の生成確率...,NaN,NaN,NaN,NaN,NaN,NaN
2,OTHER,不明,student,2,3,3,3,3,4,3,...,4,2,5,大規模言語モデルの概要、今後学んでいくことの概要を認識できた。,今後の講義を外観できた。,NaN,NaN,NaN,初学者のため、理解は薄いと思いますが、今後の講義の全体感をつかめました。引き続き、よろしくお...,NaN
3,EDUCATION,学生,student,2,4,4,2,2,3,4,...,4,5,7,LLMの大まかな構造とこれからやっていくこと、LLM が世間ではどのように実装され実践的に活...,NaN,NaN,NaN,NaN,NaN,NaN
4,SPONSOR,会員企業,student,2,2,3,3,3,3,3,...,2,2,2,・講座概要\n・LLMの概要\n └次の文字列を確率的に計算しているモデルである\n └これ...,A先生の講義,各回の概要について。全体像を掴めたのは良かったが、1時間超かけてやるべきかは議論の余地がある...,NaN,見逃しているだけだと思うが、講義内容は知識としての内容が多い印象なので、実際のアプリケーショ...,NaN,NaN


In [50]:
comments_columns = ["17", "18", "19", "21", "22"]
comments_cell = df[comments_columns].values.flatten()
comments = [v for v in comments_cell if pd.notnull(v)]
len(comments)

2296

# geminiのAPIキーを用いる

In [51]:
api_key = os.environ.get("GOOGLE_API_KEY")

### 分類

In [52]:
positive_comments = []
positive_counts = 0
negative_comments = []
negative_counts = 0
neutral_counts = 0
topic_counts = {}
important_comments = []
dangerous_comments = []

for i in range (df.shape[0]//300 + 1):
    
    comments_batch = comments[i*300:(i+1)*300]
    if len(comments_batch) == 0:
        continue

    results = classify_comments(comments_batch)

    if results and 'classifications' in results:
        for classification in results['classifications']:
            # sentimentの値と、元のコメントのインデックス番号を取得
            sentiment = classification['sentiment']
            topic = classification['topic']
            comment_index = classification['comment_index']
            
            # 元のコメントを取得
            original_comment = comments_batch[comment_index]

            # sentimentの値に応じて、各リストにコメントを追加
            if sentiment == 0:  
                positive_comments.append(original_comment)
                positive_counts += 1
            elif sentiment == 1:  
                negative_comments.append(original_comment)
                negative_counts += 1
            else:
                neutral_counts += 1

            topic_counts[topic] = topic_counts.get(topic, 0) + 1

    if results and 'important_comments' in results:
        for i in results['important_comments']:
            important_comments.append(comments_batch[i])

    if results and 'dangerous_comments' in results:
        for i in results['dangerous_comments']:
            dangerous_comments.append(comments_batch[i])


    
print("--- 分析結果の例 ---")
print(json.dumps(results, indent=2, ensure_ascii=False))
print(f'positive comments:{positive_comments}')
print(f'negative comments:{negative_comments}')
print(f'important_commnts:{important_comments}')
print(f'dangerous_comments:{dangerous_comments}')
print(topic_counts)



--- 分析結果の例 ---
{
  "classifications": [
    {
      "comment_index": 0,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 1,
      "sentiment": 0,
      "topic": 3
    },
    {
      "comment_index": 2,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 3,
      "sentiment": 2,
      "topic": 5
    },
    {
      "comment_index": 4,
      "sentiment": 0,
      "topic": 3
    },
    {
      "comment_index": 5,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 6,
      "sentiment": 1,
      "topic": 1
    },
    {
      "comment_index": 7,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 8,
      "sentiment": 2,
      "topic": 5
    },
    {
      "comment_index": 9,
      "sentiment": 2,
      "topic": 4
    },
    {
      "comment_index": 10,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 11,
      "sentiment": 1,
      "topic": 0
    },
    {
      "comment_i

### posi,negaの集約

In [62]:
summaries = summarize_comments(positive_comments, negative_comments)

# --- 結果をきれいに表示 ---
if summaries:
    print("✅ ポジティブなコメントの要約:")
    for item in summaries.get("positive_summary", []):
        print(f"  - {item}")

    print("\n" + "="*40 + "\n")

    print("❌ ネガティブなコメントの要約:")
    for item in summaries.get("negative_summary", []):
        print(f"  - {item}")

✅ ポジティブなコメントの要約:
  - LLMの全体像と歴史的背景、最新の動向を分かりやすく説明してくれた
  - 各講義の概要を事前に説明することで、今後の学習計画を立てやすくなった
  - 具体的な事例や図表が多く使用され、理解を深めるのに役立った
  - 講師の話し方が聞きやすく、テンポも適切だった
  - 質疑応答のチャットボットシステムが便利で、多くの質問に回答されていた


❌ ネガティブなコメントの要約:
  - 講義時間が長く、途中で集中力が途切れたため、休憩時間が必要
  - 専門用語が多く、事前知識がないと理解が難しい部分があった
  - 資料と講義内容に差異があった
  - OmniCampusやSlackの情報が散らばっており、使いにくい
  - チャットボットのUI/UXの改善が必要（検索機能、質問の分類、回答の精度など）


### 重要コメントの集約

In [70]:

def summarize_important_comments(comments: list[dict]) -> dict | None:
    """
    重要コメントのリストを受け取り、優先順位付けされた要約を返す関数
    """
    if not comments:
        print("要約する重要コメントがありません。")
        return None

    prompt_template = """
    あなたは、大量の顧客フィードバックを分析し、改善のためのアクションアイテムを優先順位付けする、経験豊富なプロダクトマネージャーです。

これから、重要だと判断されたコメントのリストをJSON形式で受け取ります。あなたのタスクは、これらのコメントを総合的にレビューし、最も対応すべき優先度の高い課題をトップ5まで特定し、要約してランキング形式で出力することです。

# 分析と優先順位付けのルール
1.  **内容の理解とグルーピング**: まず、全てのコメントを読み、内容の似ているもの（例：「音声が途切れる」「音が聞こえない」）を心の中でグループ化します。
2.  **優先度の決定**: 以下の基準に従って、各グループの優先度を決定してください。
    * **優先度1 (最高)**: **緊急性の高い問題**。サービスの利用を妨げる致命的な障害や、重大な間違いの指摘（例：「動画が再生できない」「ログインできない」）。
    * **優先度2 (高)**: **頻出の意見・要望**。複数のユーザーから繰り返し指摘されている問題や改善案（例：「もっと演習時間が欲しい」という声が多数）。
    * **優先度3 (中)**: **具体的な改善提案**。たとえ一人からの意見でも、具体的で実行可能な改善案（例：「各章の最後にまとめのスライドを追加してほしい」）。
    * **優先度4 (低)**: 上記以外の一般的な意見。

# 出力フォーマット
- 分析結果を以下のJSONフォーマットで出力してください。
- `priority`は1から始まる整数、`summary`はその課題や提案を簡潔にまとめた一文です。
- `indices`には、その要約の根拠となった元のコメントのインデックス番号をリストで含めてください。
- 全体で最大5つの項目を、優先度の高い順に並べてください。
- あなたの応答は、解説や前置きなしに、JSONオブジェクトのみにしてください。

{
  "ranked_summary": [
    {
      "priority": 1,
      "summary": "動画の再生に関する技術的な問題が発生しており、緊急の対応が必要です。",
      "indices": [2, 18, 45]
    },
    {
      "priority": 2,
      "summary": "講義の進行速度が速すぎるため、ついていけないという意見が多数寄せられています。",
      "indices": [6, 11, 23, 31]
    },
    {
      "priority": 3,
      "summary": "演習問題の解答例を提供してほしいという具体的な要望があります。",
      "indices": [22]
    }
  ]
}

---
それでは、以下の重要コメントリストを分析し、優先順位をつけて要約してください。
【本番】
"""
    comment_list_str = json.dumps(comments, ensure_ascii=False)

    # プレースホルダーを置換
    # f-stringを使うと{}のエスケープが面倒なので、ここでは .format() を使います
    # プロンプト内の例示JSONの { と } は {{ }} にエスケープしておく必要があります
    prompt = prompt_template + comment_list_str

    try:
        response = model.generate_content(prompt)
        clean_response = response.text.strip().replace('```json', '').replace('```', '').strip()
        summary = json.loads(clean_response)
        return summary
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None

ranked_summary = summarize_important_comments(important_comments)


if ranked_summary:
    print("--- 優先度順の要約 ---")
    for item in ranked_summary.get("ranked_summary", []):
        print(f"優先度 {item['priority']}: {item['summary']} (関連コメントNo: {item['indices']})")

--- 優先度順の要約 ---
優先度 1: Zoomリンクの共有方法やSlackの情報整理、アンケートの改善など、受講者体験を阻害する複数のUI/UX上の問題が報告されている。 (関連コメントNo: [0, 48, 50, 51, 52, 55, 75, 78, 80, 85])
優先度 2: 事前資料の提供時期や量、専門用語の多さ、講義スピードなど、学習内容の理解を妨げる複数の要因が指摘されている。 (関連コメントNo: [3, 14, 17, 26, 71, 79, 81])
優先度 3: 字幕表示機能の追加など、聴覚障がい者を含む全ての受講者にとってアクセシビリティを向上させる必要がある。 (関連コメントNo: [53])
優先度 4: 資料投影画面の録画提供、演習問題の解答例提供など、学習効果を高めるための具体的な改善要望が複数寄せられている。 (関連コメントNo: [4, 21])
優先度 4: 質疑応答の機能改善（いいね機能追加、キーワード検索機能追加など）に関する要望が複数寄せられている。 (関連コメントNo: [49, 50])
